In [102]:
import requests
from bs4 import BeautifulSoup

url = "https://www.pokepedia.fr/Liste_des_Pokémon_de_la_première_génération"
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

table = soup.find("table", class_="tableaustandard")

In [103]:
if table:

    pokemon_names = []
    pokemon_types = []
    
    rows = table.find_all("tr")[1:]  # On saute la première ligne qui contient les en-têtes

    for row in rows:
        
        cells = row.find_all("td")

        if cells:
            name_tag = cells[2].find("a")  # Prendre le <a> qui contient le nom du Pokémon
            pokemon_names.append(name_tag.text)

            types_tags = cells[7].find_all("span")  # Chercher tous les <span> contenant des types
            types = []
            for type_tag in types_tags:
                a_tag = type_tag.find("a")
                types.append(a_tag['title'].replace(" (type)", ""))
            pokemon_types.append(types)

pokemon_names
pokemon_types


[['Plante', 'Poison'],
 ['Plante', 'Poison'],
 ['Plante', 'Poison'],
 ['Feu'],
 ['Feu'],
 ['Feu', 'Vol'],
 ['Eau'],
 ['Eau'],
 ['Eau'],
 ['Insecte'],
 ['Insecte'],
 ['Insecte', 'Vol'],
 ['Insecte', 'Poison'],
 ['Insecte', 'Poison'],
 ['Insecte', 'Poison'],
 ['Normal', 'Vol'],
 ['Normal', 'Vol'],
 ['Normal', 'Vol'],
 ['Normal'],
 ['Normal'],
 ['Normal', 'Vol'],
 ['Normal', 'Vol'],
 ['Poison'],
 ['Poison'],
 ['Électrik'],
 ['Électrik'],
 ['Sol'],
 ['Sol'],
 ['Poison'],
 ['Poison'],
 ['Poison', 'Sol'],
 ['Poison'],
 ['Poison'],
 ['Poison', 'Sol'],
 ['Fée'],
 ['Fée'],
 ['Feu'],
 ['Feu'],
 ['Normal', 'Fée'],
 ['Normal', 'Fée'],
 ['Poison', 'Vol'],
 ['Poison', 'Vol'],
 ['Plante', 'Poison'],
 ['Plante', 'Poison'],
 ['Plante', 'Poison'],
 ['Insecte', 'Plante'],
 ['Insecte', 'Plante'],
 ['Insecte', 'Poison'],
 ['Insecte', 'Poison'],
 ['Sol'],
 ['Sol'],
 ['Normal'],
 ['Normal'],
 ['Eau'],
 ['Eau'],
 ['Combat'],
 ['Combat'],
 ['Feu'],
 ['Feu'],
 ['Eau'],
 ['Eau'],
 ['Eau', 'Combat'],
 ['Psy'],
 [

In [104]:
#Récuperer les noms en anglais : 
pokemon_names_anglais = []

rows = table.find_all("tr")[1:]  # On saute la première ligne qui contient les en-têtes

for row in rows:
        
    cells = row.find_all("td")

    if cells:
        name_tag = cells[3].find("a")  # Prendre le <a> qui contient le nom du Pokémon
        pokemon_names_anglais.append(name_tag.text)

pokemon_names_anglais


['Bulbasaur',
 'Ivysaur',
 'Venusaur',
 'Charmander',
 'Charmeleon',
 'Charizard',
 'Squirtle',
 'Wartortle',
 'Blastoise',
 'Caterpie',
 'Metapod',
 'Butterfree',
 'Weedle',
 'Kakuna',
 'Beedrill',
 'Pidgey',
 'Pidgeotto',
 'Pidgeot',
 'Rattata',
 'Raticate',
 'Spearow',
 'Fearow',
 'Ekans',
 'Arbok',
 'Pikachu',
 'Raichu',
 'Sandshrew',
 'Sandslash',
 'Nidoran♀',
 'Nidorina',
 'Nidoqueen',
 'Nidoran♂',
 'Nidorino',
 'Nidoking',
 'Clefairy',
 'Clefable',
 'Vulpix',
 'Ninetales',
 'Jigglypuff',
 'Wigglytuff',
 'Zubat',
 'Golbat',
 'Oddish',
 'Gloom',
 'Vileplume',
 'Paras',
 'Parasect',
 'Venonat',
 'Venomoth',
 'Diglett',
 'Dugtrio',
 'Meowth',
 'Persian',
 'Psyduck',
 'Golduck',
 'Mankey',
 'Primeape',
 'Growlithe',
 'Arcanine',
 'Poliwag',
 'Poliwhirl',
 'Poliwrath',
 'Abra',
 'Kadabra',
 'Alakazam',
 'Machop',
 'Machoke',
 'Machamp',
 'Bellsprout',
 'Weepinbell',
 'Victreebel',
 'Tentacool',
 'Tentacruel',
 'Geodude',
 'Graveler',
 'Golem',
 'Ponyta',
 'Rapidash',
 'Slowpoke',
 'Sl

In [105]:
def get_all_pages():
    urls =[]

    for k in range(len(pokemon_names)):
        k = f"https://www.pokepedia.fr/{pokemon_names[k]}"
        urls.append(k)
    return urls

urls = get_all_pages()

Maintenant il va falloir récupérer les statistiques de chaque pokémon :

In [106]:
def get_pokemon_stats(pokemon_name):
    url = f"https://www.pokepedia.fr/{pokemon_name}"

    response = requests.get(url)

    if response.status_code != 200:
        print(f"Pas d'accès possible pour {pokemon_name}")
        return None

    soup = BeautifulSoup(response.text, "html.parser")

    index = pokemon_names.index(pokemon_name)


    # Trouver le tableau des statistiques
    stats_table = soup.find("table", class_=f"tableaustandard {pokemon_types[index][0].lower()} tableau-overflow")
    if not stats_table:
        print(f"Pas de tableau de stats trouvé pour {pokemon_name}")
        return None

    # Extraire les statistiques (PV, Attaque, Défense, Vitesse)
    rows = stats_table.find_all("tr")

    stats = {"Nom" : pokemon_name}
    stat_names = ["PV", "Attaque", "Attaque Spéciale", "Défense Spéciale", "Défense", "Vitesse"]  # Stats à récupérer

    for row in rows:
        cols = row.find_all("td")

        if len(cols) >= 2:
            stat_label = cols[0].text.strip()  # Texte de la première colonne (nom de la stat)
            stat_value = cols[1].text.strip()  # Valeur de la deuxième colonne

            if "PV" in stat_label:
                stat_label = "PV"
 
            # Vérifier si c'est bien l'une des statistiques recherchées
            if stat_label in stat_names:
                stats[stat_label] = stat_value

    # Vérifier si on a bien trouvé toutes les stats, sinon mettre "N/A"
    for stat in stat_names:
        if stat not in stats:
            stats[stat] = "N/A"

    return stats # Retourner les statistiques sous forme de dictionnaire

# 🔍 Test avec Bulbizarre
pokemon_name = "Bulbizarre"
stats = get_pokemon_stats(pokemon_name)

if stats:
    print(f"📊 Statistiques de {pokemon_name} :", stats)

📊 Statistiques de Bulbizarre : {'Nom': 'Bulbizarre', 'PV': '45', 'Attaque': '49', 'Défense': '49', 'Attaque Spéciale': '65', 'Défense Spéciale': '65', 'Vitesse': '45'}


Création du dataframe :

In [107]:
import pandas as pd 

df_gen1 = pd.DataFrame({
    "Nom": pokemon_names,
    "Type": pokemon_types,
    "Géneration": ["1"] * len(pokemon_names)
})

stats_list = []
for name in df_gen1["Nom"]:
    stats = get_pokemon_stats(name)  # Récupérer les stats sous forme de dictionnaire
    if stats:
        stats_list.append(stats)

df_stats =pd.DataFrame(stats_list)

df_gen1 = df_gen1.merge(df_stats, on="Nom", how="left")

In [108]:
df_gen1

,Nom,Type,Géneration,PV,Attaque,Défense,Attaque Spéciale,Défense Spéciale,Vitesse
0,Bulbizarre,"[Plante, Poison]",1,45,49,49,65,65,45
1,Herbizarre,"[Plante, Poison]",1,60,62,63,80,80,60
2,Florizarre,"[Plante, Poison]",1,80,82,83,100,100,80
3,Salamèche,[Feu],1,39,52,43,60,50,65
4,Reptincel,[Feu],1,58,64,58,80,65,80
...,...,...,...,...,...,...,...,...,...
146,Minidraco,[Dragon],1,41,64,45,50,50,50
147,Draco,[Dragon],1,61,84,65,70,70,70
148,Dracolosse,"[Dragon, Vol]",1,91,134,95,100,100,80
149,Mewtwo,[Psy],1,106,110,90,154,90,130


In [109]:
df_gen1.to_csv("pokemon_gen1.csv", encoding="utf-8")


In [110]:
df_gen1["Names"] = pokemon_names_anglais
df_gen1

,Nom,Type,Géneration,PV,Attaque,Défense,Attaque Spéciale,Défense Spéciale,Vitesse,Names
0,Bulbizarre,"[Plante, Poison]",1,45,49,49,65,65,45,Bulbasaur
1,Herbizarre,"[Plante, Poison]",1,60,62,63,80,80,60,Ivysaur
2,Florizarre,"[Plante, Poison]",1,80,82,83,100,100,80,Venusaur
3,Salamèche,[Feu],1,39,52,43,60,50,65,Charmander
4,Reptincel,[Feu],1,58,64,58,80,65,80,Charmeleon
...,...,...,...,...,...,...,...,...,...,...
146,Minidraco,[Dragon],1,41,64,45,50,50,50,Dratini
147,Draco,[Dragon],1,61,84,65,70,70,70,Dragonair
148,Dracolosse,"[Dragon, Vol]",1,91,134,95,100,100,80,Dragonite
149,Mewtwo,[Psy],1,106,110,90,154,90,130,Mewtwo


In [111]:
df_gen1 =df_gen1[["Names", "Nom", "Type", "PV", "Attaque", "Attaque Spéciale", "Défense", "Défense Spéciale", "Vitesse", "Géneration"]]
df_gen1 = df_gen1.rename(columns={"Nom": "Noms", "Géneration": "Génération"})


In [116]:
cols_numeriques = ["PV", "Attaque", "Attaque Spéciale", "Défense", "Défense Spéciale", "Vitesse"]

df_gen1[cols_numeriques] = df_gen1[cols_numeriques].apply(pd.to_numeric, errors="coerce")

df_gen1["Somme Stats"] = df_gen1[cols_numeriques].sum(axis=1)

df_gen1["Moyenne Statistique"] = (df_gen1["Somme Stats"] / 6).round(2)

In [114]:
df_gen1

,Names,Noms,Type,PV,Attaque,Attaque Spéciale,Défense,Défense Spéciale,Vitesse,Génération,Somme Stats,Moyenne Statistique
0,Bulbasaur,Bulbizarre,"[Plante, Poison]",45,49,65,49,65,45,1,318,53.00
1,Ivysaur,Herbizarre,"[Plante, Poison]",60,62,80,63,80,60,1,405,67.50
2,Venusaur,Florizarre,"[Plante, Poison]",80,82,100,83,100,80,1,525,87.50
3,Charmander,Salamèche,[Feu],39,52,60,43,50,65,1,309,51.50
4,Charmeleon,Reptincel,[Feu],58,64,80,58,65,80,1,405,67.50
...,...,...,...,...,...,...,...,...,...,...,...,...
146,Dratini,Minidraco,[Dragon],41,64,50,45,50,50,1,300,50.00
147,Dragonair,Draco,[Dragon],61,84,70,65,70,70,1,420,70.00
148,Dragonite,Dracolosse,"[Dragon, Vol]",91,134,100,95,100,80,1,600,100.00
149,Mewtwo,Mewtwo,[Psy],106,110,154,90,90,130,1,680,113.33


In [117]:
df_gen1.to_csv("pokemon_gen1.csv", encoding="utf-8")